In [ ]:
from pystac_client.client import Client
import geopandas as gpd
import rioxarray as riox
import json

from rasterio.session import AWSSession
import rasterio

In [ ]:
# use the creds created on CDSE website
with open("/data/creds_s3.json") as f:
    cred = json.load(f)

un = cred["username"]
pw = cred["password"]

# rasterio handles S3 sessions
session = AWSSession(
    aws_access_key_id=un,
    aws_secret_access_key=pw,
    region_name="default",
    endpoint_url="eodata.dataspace.copernicus.eu")

In [ ]:
# Search using STAC api
catalog = Client.open(
    "https://stac.dataspace.copernicus.eu/v1/"
)
aoi_file = "../data/Morocco_AOI.geojson"
shp = gpd.read_file(aoi_file).geometry[0]
search = catalog.search(collections=["sentinel-1-slc"], intersects=shp)
it = next(search.items())
url = it.assets["iw1-vv"].href

In [ ]:
# Use rasterio.Env to pass the custom session:
with rasterio.Env(session=session, AWS_VIRTUAL_HOSTING=False):
    ds = riox.open_rasterio(url, chunks="auto")

In [ ]:
# ds[:100].compute()